# PINN for Uniform Rectilinear Motion

# Importing Necessary Packages

In [19]:
import torch
import torch.nn as nn
from torch.func import functional_call, grad, vmap
import torchopt

## Building the Network's Architecture

In the following cell, we define the archictecture's hyperparameters, note that the activation function applied here is a hyperbolic tangent (Tanh) for it being non-linear and ranging from -1 to 1. It is also interesting to pay attention to the fact that we are building the linear neural network from scratch due the lack of a PyTorch module specific for it, i.e., we are building a custom module.

In [20]:
class LinearNN(nn.Module):
    def __init__(
            self,
            num_inputs: int=1,
            num_layers: int=1,
            num_neurons: int=5,
            act: nn.Module = nn.Tanh()
    ) -> None:
        super().__init__()
        self.num_inputs = num_inputs
        self.num_neurons = num_neurons
        self.num_layers = num_layers

        layers = []

        # Input layer
        layers.append(nn.Linear(self.num_inputs, num_neurons))

        # Hidden layers
        for _ in range(num_layers):
            layers.extend([nn.Linear(num_neurons, num_neurons), act])

        # Output layers
        layers.append(nn.Linear(num_neurons, 1))

        # Building the network as sequential
        self.network = nn.Sequential(*layers)

    # Setting up the output
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.network(x.reshape(-1,1)).squeeze()

## Determining Loss Function

Below a Python function is developed to facilitate the development of our mathematical function, which afterwards will be used to define the network's loss function.   

Given we are dealing with a Uniform Rectilinear Motion (URM), the differential equation is given by:

\begin{equation}
    \dfrac{ds}{dt} - v = 0,
\end{equation}

where $s$ refers to a temporal function of space, $t$ to time and $v$ to a constant velocity. Basically, this equation describes the movement of a body with constant velocity.

In order to make it easier to generalize our model, the equation above will be rewritten as:

\begin{equation}
    \dfrac{df}{dx} - c = 0
\end{equation}

In [21]:
# Running the structure created
model = LinearNN()

# Python function to build mathematical functions
def f(x: torch.Tensor, params: dict[str, torch.nn.Parameter]) -> torch.Tensor:
    return functional_call(model, params, (x, ))

# Mathematical function
dsdx = vmap(grad(f), in_dims=(0, None))

In [22]:
# Setting the boundaries values
## Space boundary
S_BOUNDARY = 0.0
## Velocity boundary
C_BOUNDARY = 0.5

def loss_fn(params: torch.Tensor, x: torch.Tensor):

    # Interior equation
    interior = dsdx(x, params) - C_BOUNDARY
    
    # Parametrizing boundary
    s_boundary = torch.Tensor([S_BOUNDARY])
    c_boundary = torch.Tensor([C_BOUNDARY])
    
    # Boundary equation
    boundary = f(s_boundary, params) - c_boundary 

    # Setting Mean Squared Error as metric for loss
    loss = nn.MSELoss()
    # Getting numerical value for loss
    loss_value = loss(interior, torch.zeros_like(interior)) + loss(boundary, torch.zeros_like(boundary))

    return loss_value

In [23]:
batch_size = 30
num_iter = 100
learning_rate = 1e-1
domain = (-5.0,5.0)

optmizer = torchopt.FuncOptimizer(torchopt.adam(lr=learning_rate))

params = tuple(model.parameters())

for i in range(num_iter):

    x = torch.FloatTensor(batch_size).uniform_(domain[0], domain[1])

    loss = loss_fn(params, x)

    params = optmizer.setp(loss, params)

    print(f"Iteration {i} with loss {float(loss)}") 

ValueError: Expected all elements of parameter_and_buffer_dicts to be dictionaries